In [ ]:
include("_.jl")

In [ ]:
an_ge_ = Dict{String,Vector{String}}()

In [ ]:
hu_ = convert(Vector{String}, kwat.gene.select(ba_ = []))

## Mouse Genome Informatics

In [ ]:
hu_mo = convert(Dict{String,String}, kwat.gene.map_mouse()[2])

println(length(hu_mo))

an_ge_["Human Genes based on MGI"] = [hu for hu in hu_ if !haskey(hu_mo, hu)]

## 2011 De Novo Origin of Human Protein-Coding Genes

In [ ]:
en_ = readlines(joinpath(PAI, "De Novo Origin of Human Protein-Coding Genes", "genes.txt"))

an_ge_["Human Genes based on 2011 paper"] =
    [["CLLU1", "c22orf45", "DNAH10OS"]; convert(Vector{String}, kwat.gene.rename(en_)[1])]

## 2019 Genes with human-specific features are primarily involved with brain, immune and metabolic evolution

In [ ]:
an_ge_ir = read_xlsx(
    joinpath(
        PAI,
        "Genes with human-specific features are primarily involved with brain, immune and metabolic evolution",
        "12859_2019_2886_MOESM2_ESM.xlsx",
    ),
    "HumanSpecific Genes",
)

an_ge_["Human Genes based on 2019 paper"] = string.(an_ge_ir[!, 1])

## Write

In [ ]:
DictExtension.write(joinpath(PAO, "analysis_to_genes.json"), sort(an_ge_))